# Preprocessing data for the Data Challenge

In [1]:
!pip install langdetect --quiet

In [2]:
!pip install googletrans --quiet

In [3]:
!pip install google_trans_new --quiet

In [4]:
!pip install deep_translator --quiet

In [5]:
import pandas as pd
import numpy as np
import nltk


import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

import time
import re
from langdetect import detect
from googletrans import Translator
from google_trans_new import google_translator 

from collections import Counter

import gc

from deep_translator import (GoogleTranslator)


Bad key text.latex.preview in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You proba

In [6]:
FILENAME = "../data/train_series.csv"
FILENAME_ECB = "../data/ecb_data.csv"
FILENAME_FED = "../data/fed_data.csv"
FILENAME_TRANS = "../data/translated_data.csv"

In [7]:
ecb = pd.read_csv(FILENAME_ECB, index_col=0)
fed = pd.read_csv(FILENAME_FED, index_col=0)
#df_translated = pd.read_csv(FILENAME_TRANS, index_col=0)

In [8]:
ecb.head(1)

,title,speaker,text
0,Comments by Yves Mersch at Financial Services ...,Yves Mersch,Comments by Yves Mersch at Financial Service...


In [9]:
fed.head(1)

,title,speaker,text
0,The Importance of Economic Education and Finan...,Governor Frederic S. Mishkin,As ...


## Apply the pipeline for english text

In [10]:
# Before dealing with N/A because otherwise, this edit would just undo the other.
# We can use this to remove the "TRANSCRIPT" and "SPEECH" tags.
# We also notice that most of the time, the content is preceded by a date. It would be useful to make a date parser to remove the first chunk.

def numbered_reference_removal(text):
    # refs are typically in the form [n] in the text.
    matches = re.findall(r'\[[0-9]+?\]', text)
    counter = Counter(matches)
    for match in matches:
        if counter[match] != 2:
            # print(f"Didn't find a reference twice in the text, but {counter[match]} times. Cannot remove.")
            # print(text[:2000], '\n ... \n',text[-2000:])
            # print()
            return text
    
    assert len(matches)%2 == 0
    if len(matches) == 0:
        return text
    N = len(matches)//2
    is_ordered = True
    n = 0
    for s in matches:
        is_ordered = (s == f'[{n%N+1}]')
        if not is_ordered : 
            # print(f"Not ordered, {str(matches)}")
            return text
        n += 1
    
    # Remove all references after [1]
    res = re.findall(
        r'^.*?\[1\].+?\[1\]',
        text
    )
    return res[0]

def reference_removal_en(text):
    res = re.sub(r'(?i)see also.*', '', text)
    res = re.sub(r'References.*', '', res)
    res = re.sub("\[[0-9]+\]", "", res)
    return res

def first_date_extractor(text):
    if len(text) > 0:
        res = re.sub('^(.*?)[1-9][0-9]* (?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|'
                     + 'Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?) (19|20)[0-9][0-9]',
                     '', text)
        return res

def remove_title(x):
    if not x["text"] is None and isinstance(x["text"], str):
        res = re.sub(x["title"], '', x["text"]).strip()
        return res
    else:
        return x["title"]

def website_remover(text):
    # Remove websites
    regex = "((http|https)://)(www.)?" \
        + "[a-zA-Z0-9@:%._\\+~#?&//=]{2,256}\\.[a-z]" \
        + "{2,6}\\b([-a-zA-Z0-9@:%._\\+~#?&//=]*)"
    res = re.sub(regex, "", text).strip()
    return res

def tag_removal(text):
    # Remove tags
    res = re.sub('SPEECH', '', text)
    res = re.sub('TRANSCRIPT', '', res)
    res = re.sub("Introduction", "", res)
    res = re.sub("Summary", "", res)
    return res

def pipeline_en(x):
    res = remove_title(x)
    if res is None:
        return res
    res = numbered_reference_removal(res)
    res = reference_removal_en(res)
    res = tag_removal(res).strip()
    res = first_date_extractor(res).strip()

    # print("================================================")
    # print(res[:200])
    # print("...")
    # print(res[-500:])
    return res

ecb["text_preprocessed"] = ecb.apply(pipeline_en, axis=1)

In [11]:
#ecb["text_preprocessed"][31]

## Detect the text in english, deutsch, french, spanish or italian

In [12]:
ecb["lang"] = ecb["text_preprocessed"].apply(lambda x : detect(x[:500]))
fed["lang"] = fed["text"].apply(lambda x : detect(x[:500]))

In [13]:
ecb["lang"].value_counts()

en    1646
de      75
fr      31
es      16
it       4
Name: lang, dtype: int64

In [14]:
fed["lang"].value_counts()

en    739
Name: lang, dtype: int64

In [15]:
ecb.columns

Index(['title', 'speaker', 'text', 'text_preprocessed', 'lang'], dtype='object')

In [16]:
'''
Calculer la somme de la longueur de la chaîne de caractères de cette colonne pour 
toutes les lignes qui ne sont pas en langue anglaise.
L'argument skipna=True de la méthode sum() indique que les valeurs manquantes (NaN) 
doivent être ignorées dans le calcul de la somme
'''
ecb[ecb["lang"] != "en"]["text_preprocessed"].str.len().sum(skipna=True)

2565448

In [17]:
num_words_total = ecb["text_preprocessed"].str.split().apply(len).sum(skipna=True)
print("Au total, le premier texte contient {} mots.".format(num_words_total))

Au total, le premier texte contient 4610038 mots.


In [18]:
first_text = ecb[ecb["lang"] != "en"].iloc[0]["text_preprocessed"]
num_words_first_line = len(first_text.split())
print("Le premier texte contient {} mots.".format(num_words_first_line))

Le premier texte contient 2573 mots.


## Split all texts in a list of texts of 4500 characters maximum

In [42]:
italian_text = ecb[ecb["lang"] == "it"]["text_preprocessed"]
english_text = ecb[ecb["lang"] == "en"]["text_preprocessed"]
spanish_text = ecb[ecb["lang"] == "es"]["text_preprocessed"]
french_text = ecb[ecb["lang"] == "fr"]["text_preprocessed"]
deutsch_text = ecb[ecb["lang"] == "de"]["text_preprocessed"]
spanish_text.index

Int64Index([ 185,  301,  349,  353,  358,  375,  589,  698,  830, 1164, 1205,
            1458, 1468, 1489, 1499, 1517],
           dtype='int64')

In [20]:
def split_text(text):
    nltk.download('punkt')
    
    sentences = nltk.sent_tokenize(text)
    new_text = []
    current_text = ''
    for sentence in sentences:
        if len(current_text + sentence) <= 4500:
            current_text += sentence
        else:
            if current_text[-1] != '.':
                last_period = current_text.rfind('.')
                if last_period != -1:
                    new_text.append(current_text[:last_period+1])
                    current_text = current_text[last_period+1:]
            new_text.append(current_text)
            current_text = sentence
    if current_text != '':
        new_text.append(current_text)
    
    
    if len(new_text[-1]) < 200 and len(new_text) > 1:
        new_text[-2] = new_text[-2] + new_text[-1]
        new_text.pop(-1)
    
    return new_text

In [21]:
def df_with_split_text(df):
    nltk.download('punkt')
    df_split_text = df.copy()

    for index in range(df_split_text.shape[0]):
        new_text = split_text(df_split_text.iloc[index]['text_preprocessed'])
        df_split_text.iloc[index]['text_preprocessed'] = new_text

    return df_split_text

In [22]:
ecb_split_text = df_with_split_text(ecb)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-t

In [23]:
len(ecb_split_text.iloc[0]['text_preprocessed'])

2

In [24]:
ecb_split_text['text_preprocessed'].isna().sum()

0

## Translate all text in english

In [25]:
from langdetect import detect

In [75]:
def translate_text(text, index):
    translated_text = text
    
    if index in deutsch_text.index : 
        lang_text = 'german'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index in italian_text.index : 
        lang_text = 'it'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index in spanish_text.index : 
        lang_text = 'es'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
        
    if index in french_text.index : 
        lang_text = 'fr'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index in english_text.index:
        translated_text = text
    
    return translated_text

In [76]:
def translate_df(df):
    
    df_translated = df.copy()

    for index in range(df_translated.shape[0]):
        list_translated_text = []
        for i in range(len(df_translated.iloc[index]['text_preprocessed'])):
            translated_text_i = translate_text(df_translated.iloc[index]['text_preprocessed'][i], index)
            list_translated_text.append(translated_text_i)
        df_translated.iloc[index]['text_preprocessed'] = list_translated_text

    return df_translated

In [79]:
df_translated_text = translate_df(ecb_split_text)
df_translated_text.iloc[31]['text_preprocessed']


['?Speech by Jörg Asmussen, Member of the Governing Board of the ECB, Handelsblatt Annual Conference “Banks in Transition”, Frankfurt am Main, September 4, 2012 Dear Sir or Madam, A sentence in the final declaration of the euro summit on June 28 of this year caused concern for people to sit up and take notice and plenty to talk about over the summer - I quote: "Once an effective single supervisory mechanism for euro area banks has been put in place, involving the ECB, the ESM would, following an orderly decision, have the option to recapitalize banks directly." Here was the start of a financial market union was decided. As a contribution to this discussion, I would like to address three aspects below: Where do we stand with financial market regulation? What is the current situation on the financial markets? Why has a financial market union become necessary? What should it look like? What role can the ECB play within the new supervisory architecture, what condition ments have to be met?

In [ ]:
df_translated_text.iloc[31]['text_preprocessed']

In [81]:
df_translated_text['text_preprocessed'] = df_translated_text['text_preprocessed'].apply(lambda x: ''.join(x))

In [82]:
df_translated_text.to_csv('translated_data.csv', index=False)

In [83]:
FILENAME = "../data/translated_data.csv"
translated_data = pd.read_csv(FILENAME, index_col=0)

In [86]:
translated_data.iloc[31]['text_preprocessed']

'?Speech by Jörg Asmussen, Member of the Governing Board of the ECB, Handelsblatt Annual Conference “Banks in Transition”, Frankfurt am Main, September 4, 2012 Dear Sir or Madam, A sentence in the final declaration of the euro summit on June 28 of this year caused concern for people to sit up and take notice and plenty to talk about over the summer - I quote: "Once an effective single supervisory mechanism for euro area banks has been put in place, involving the ECB, the ESM would, following an orderly decision, have the option to recapitalize banks directly." Here was the start of a financial market union was decided. As a contribution to this discussion, I would like to address three aspects below: Where do we stand with financial market regulation? What is the current situation on the financial markets? Why has a financial market union become necessary? What should it look like? What role can the ECB play within the new supervisory architecture, what condition ments have to be met? 